__Some technical details first. Let's import the python packages we would need.__

In [ ]:
import spacy
import nltk
import os
import string
import re

import pandas as pd
import pickle

from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

import shutil
from shutil import copyfile, copy
from sys import exit

In [ ]:
#Set some paths up
temp_path = 'C:/Project/temp/'                  #Path for temporary files created
corpus_path = 'C:/Project/corpus/'              #Path to corpus

# Interaction with the User

### The search query -  It comes from the user. Go ahead, type your search.

In [ ]:
#Run this cell, type in your search query and then press Enter.
the_query = input("Hi, what would you like to search for today? :")

In [ ]:
#Enter a path for your results
print("Great! Where would you like the results to be stored?")
pathexists = 0
while pathexists == 0:
    cache = input("Enter a valid path: ")
    if os.path.isdir(cache) == True:
        print('Your results will be stored in',cache)
        pathexists = 1
        break
    else:
        print("The path doesn't exist. Try again!")

# Natural Language Processing

The question text will now be passed through the NL processor to ascertain what's being asked.

In [ ]:
def nlp_proc(the_query):
    spacy_nlp = spacy.load('en_core_web_sm')
    doc = spacy_nlp(the_query)
    relevant_pos = []
    stop = set(stopwords.words('english'))
    for token in doc:
        #print(token.text, token.pos_, token.tag_, token.dep_,token.shape_)
        if token.pos_ in ['NOUN','VERB','ADJ'] and token.text not in stop:
            relevant_pos.append([token.text,token.pos_])
    return doc, relevant_pos

def search_term(search_list, table_name, col1, col2):
    files=[]
    for search_term in search_list:
        for file, rec in zip(table_name[col1],table_name[col2]):
            for item in rec:
                if search_term.lower() in item.strip():
                    files.append(file)
    return files

In [ ]:
spacyd_query,get_synonyms = nlp_proc(the_query)
get_synonyms

In [ ]:
entities = [ent.text for ent in spacyd_query.ents]
entities

In [ ]:
#get synonyms for relevant search terms from the query
synonyms={}
lim = 3      #increase this limit if not enough matches in the first try
for item in get_synonyms:
    tempsyn =[item[0]]
    if item[1] in 'NOUN':
        POS_eq = wn.synsets(item[0],pos=wn.NOUN)
    elif item[1] in 'VERB':
        POS_eq = wn.synsets(item[0],pos=wn.VERB)
    elif item[1] in 'ADJ':
        POS_eq = wn.synsets(item[0],pos=wn.ADJ)
    #entailments can be appended as well - see https://blog.xrds.acm.org/2017/07/power-wordnet-use-python/
    for syn in POS_eq [:lim]: 
        for l in syn.lemmas():                                   #same
            tempsyn.append(l.name()) 
        for h in syn.hypernyms():                                #similar
            for i in h.lemmas():
                tempsyn.append(i.name())
        for e in syn.entailments():                              #implications
            for j in e.lemmas():
                tempsyn.append(j.name())                
        for ph in syn.part_holonyms():                           #higher context in the hierarchy
            for k in ph.lemmas():
                tempsyn.append(k.name())                                
        for pm in syn.part_holonyms():                           #lower context in the hierarchy
            for m in pm.lemmas():
                tempsyn.append(m.name())
                
    synonyms[item[0]]=list(set(tempsyn))
synonyms

In [ ]:
#import document table
doc_table = pd.read_pickle(temp_path + "CRA_doc_table.pkl")

In [ ]:
#search entities first
docs_with_entities=[]
if len(entities) > 0:
    files = search_term(entities,doc_table,'ID','single_terms')
    if len(files) > 0:
        docs_with_entities=list(set(files))
docs_with_entities

In [ ]:
#search other terms and get counts etc.
docs_with_terms_and_synonyms = []
for list_item in get_synonyms:
    temp_list = []
    files = search_term(synonyms[list_item[0]],doc_table,'ID','single_terms')
    print(files)
    if len(files) > 0:
        if docs_with_terms_and_synonyms == []:
            docs_with_terms_and_synonyms = files
        else:
            intersection = list(set(docs_with_terms_and_synonyms) & set(files))
            if len(intersection) > 0:
                docs_with_terms_and_synonyms = intersection
status = ''
if len(docs_with_terms_and_synonyms) > 0 and len(docs_with_entities) > 0:
    intersection = list(set(docs_with_terms_and_synonyms) & set(docs_with_entities))
    if len(intersection) > 0:
        all_docs = intersection
    else:
        status = 'PARTIAL'
        if len(docs_with_terms_and_synonyms) > len(docs_with_entities):
            all_docs = docs_with_terms_and_synonyms
        else:
            all_docs = docs_with_entities
        
else:
    status = 'PARTIAL'
    all_docs = list(set(docs_with_terms_and_synonyms) | set(docs_with_entities))
if not len(get_synonyms) > 0 or not len(entities) > 0:
    status = ''
all_docs

In [ ]:
if len(all_docs) > 0:
    for filen in all_docs:
        for file, name in zip(doc_table['ID'],doc_table['file_name']):
            if file == filen:
                copy_file = name
                source = os.path.join(corpus_path, copy_file)
                dest = cache
                shutil.copy(source, dest)
else:
    status = 'NONE'

In [ ]:
print('Hello there, I am back!')
print('Your wanted to search for : ',the_query)
if status == 'NONE':
    print('Unfortunately, my search did not return any results.')
    print('This is a rare event! Or perhaps it was just a spelling error? Would you like to try again?')
else:
    if status == 'PARTIAL':
        print('Only a partial search was successful. You may want to re-word your search query to get better results.')                 
    print('You will find the documents in',cache)
    print('Let me know how it goes!')
    print('Some Information:')
    print('The following terms were used for the search:', end = " ")
    if len(synonyms) > 0:
        for item in get_synonyms: 
            print(synonyms[item[0]],end = " " )
    if len(entities) > 0:
        print('as well as these name entities:', entities)    
    print('Number of documents returned:', len(all_docs))